# Экспертный код

In [37]:
!pip install -q tavily langchain_deepseek langchain==0.3.27

In [38]:
from tavily import TavilyClient
from langchain.tools import tool
from langchain_deepseek import ChatDeepSeek
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime
import pandas as pd
import sqlite3

In [ ]:
tavily_client = TavilyClient(api_key='')
deepseek = ChatDeepSeek(
    api_key='',
    temperature = 0,
    model="deepseek-chat"
)
today = datetime.now().strftime('%d.%m.%Y')

In [40]:
DB_NAME = 'hacathon_db'

def init_db():
    """Создает таблицу, если она еще не существует"""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Создание таблицы по условию: id, question, agent_answer
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS research_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT NOT NULL,
            agent_answer TEXT
        )
    ''')

    conn.commit()
    conn.close()
init_db()


In [42]:
@tool
def search_web(query: str):
  '''
  Используй этот инструмент для поиска в интернете
  Входной параметр query: str - поисковый запрос
  '''
  return tavily_client.search(query)


@tool
def execute_sql(query: str):
  '''
  Используй этот инструмент для работы с базой sqlite3
  Входной параметр query: str - sql запрос.

  Структура созданной БД:
  CREATE TABLE IF NOT EXISTS research_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT NOT NULL,
    agent_answer TEXT
  )
  '''

  conn = sqlite3.connect(DB_NAME)
  cursor = conn.cursor()
  cursor.execute(query)
  if query.strip().upper().startswith("SELECT"):
      data = cursor.fetchall()
  else:
      data = {"rows_affected": cursor.rowcount}
  conn.commit()
  conn.close()
  return data


prompt = ChatPromptTemplate.from_messages([
    ("system", (
        f"Ты поисковый чатбот. Сегодня {today}."
        "Твоя цель — искать в интернете ответы на вопросы пользователя и давать на них ответы."
        "Ответы клади в базу данных и ответ на вопрос читай тоже из базы данных"
      )
    ),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])


agent = create_tool_calling_agent(deepseek, [search_web, execute_sql], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[search_web, execute_sql], verbose=True)
response = agent_executor.invoke({
    'input': 'Найди наикратчайщий путь от Лицея БГУ до ДС Масюковщина. Включая метро, автобусы, троллейбусы.'
})


print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': 'Лицей БГУ Минск адрес как добраться'}`
responded: Я помогу вам найти кратчайший путь от Лицея БГУ до ДС Масюковщина. Сначала поищу актуальную информацию о маршрутах общественного транспорта в Минске.

{'query': 'Лицей БГУ Минск адрес как добраться', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://dir.by/belarus/minsk/licej_bgu/index.php', 'title': 'Лицей БГУ (г. Минск, Беларусь) ул. Ульяновская, 8', 'content': 'Лицей БГУ ул. Ульяновская 8 г. Минск Беларусь Адрес: г. Минск, ул. Ульяновская, 8. Телефон: 209-52-65 (приемная). Описание. Лицей БГУ проводит: • обучение', 'score': 0.87681407, 'raw_content': None}, {'url': 'https://yandex.by/maps/org/litsei_biloruskoho_derzhavnoho_universytetu/1112678425/', 'title': 'БГУ, лицей имени Ф.Э. Дзержинского - Яндекс', 'content': 'Лицей «БГУ, лицей имени Ф.Э. Дзержинского» по адресу Минск, Ульяновская улица, 8, метро «Первомайская»